<a href="https://colab.research.google.com/github/akibkhan1/skin-lesion-classification/blob/main/color_constancy_isic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [1]:
!gdown --id 10TNZNGbTaR2sZN7tL8riGZwBzJCWmWA6

Downloading...
From: https://drive.google.com/uc?id=10TNZNGbTaR2sZN7tL8riGZwBzJCWmWA6
To: /content/ISIC-split.zip
2.77GB [00:39, 70.8MB/s]


In [ ]:
!unzip ISIC-split.zip

In [5]:
def shade_of_gray_cc(img, power=6, gamma=2.2):
    """
    img (numpy array): the original image with format of (h, w, c)
    power (int): the degree of norm, 6 is used in reference paper
    gamma (float): the value of gamma correction, 2.2 is used in reference paper
    """
    img_dtype = img.dtype

    if gamma is not None:
        img = img.astype('uint8')
        look_up_table = np.ones((256,1), dtype='uint8') * 0
        for i in range(256):
            look_up_table[i][0] = 255 * pow(i/255, 1/gamma)
        img = cv2.LUT(img, look_up_table)

    img = img.astype('float32')
    img_power = np.power(img, power)
    rgb_vec = np.power(np.mean(img_power, (0,1)), 1/power)
    rgb_norm = np.sqrt(np.sum(np.power(rgb_vec, 2.0)))
    rgb_vec = rgb_vec/rgb_norm
    rgb_vec = 1/(rgb_vec*np.sqrt(3))
    img = np.multiply(img, rgb_vec)

    # Andrew Anikin suggestion
    img = np.clip(img, a_min=0, a_max=255)
    
    return img.astype(img_dtype)

In [6]:
def apply_cc (img_paths, output_folder_path, resize=None):
    
    if not os.path.isdir(output_folder_path):
        os.mkdir(output_folder_path)    

    with tqdm(total=len(img_paths), ascii=True, ncols=100) as t:
        
        for img_path in img_paths:
            img_name = img_path.split('/')[-1]
            img_ = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if resize is not None:
                img_ = cv2.resize(img_, resize, cv2.INTER_AREA)
            np_img = shade_of_gray_cc (img_)            
            cv2.imwrite(os.path.join(output_folder_path, img_name.split('.')[0] + '.jpg'), np_img)
            t.update()

In [7]:
image_classes = {
  0: "AKIEC",
  1: "BCC",
  2: "BKL",
  3: "DF",
  4: "MEL",
  5: "NV",
  6: "VASC"
}

In [8]:
%cd /content/ISIC-full-split/training/


/content/ISIC-full-split/training


In [20]:
all_paths = []
for i in range(7):
  glob_path = "/content/ISIC-full-split/training/" + str(image_classes[i]) + "/" +"*"+ ".jpg"
  img_paths = glob(glob_path)
  all_paths.append(img_paths)

In [23]:
len(all_paths[0])

261

In [29]:
%cd /content/

/content


In [30]:
output_folder_path = "ISIC-full-split-cc/training/"

for i in range(7):
  output = output_folder_path + str(image_classes[i]) + "/"
  apply_cc (all_paths[i], output, (400,400))

100%|#############################################################| 113/113 [00:03<00:00, 32.40it/s]


In [31]:
all_paths_validation = []
for i in range(7):
  glob_path = "/content/ISIC-full-split/validation/" + str(image_classes[i]) + "/" +"*"+ ".jpg"
  img_paths = glob(glob_path)
  all_paths_validation.append(img_paths)

In [34]:
%cd /content/

/content


In [36]:
output_folder_path = "ISIC-full-split-cc/validation/"

for i in range(7):
  output = output_folder_path + str(image_classes[i]) + "/"
  apply_cc (all_paths_validation[i], output, (400,400))

100%|###############################################################| 29/29 [00:00<00:00, 32.04it/s]


In [ ]:
!zip -r ISIC-split-cc.zip ISIC-full-split-cc

In [38]:
!cp -r ISIC-split-cc.zip "/content/drive/MyDrive/Colab_Notebooks/Thesis/"